In [1]:
%matplotlib ipympl

In [2]:
import matplotlib.pyplot as plt
import matplotlib.collections
import numpy as np
from cartpole import CartPole

In [3]:
env = CartPole(visual=False)
env.sim_steps = 50         # number of Euler integration steps to perform in one go
env.delta_time = 0.05

### Small Amplitude Oscillation About Equilibrium

In [4]:
# [cart_pos, cart_vel, pole_angle, pole_angular_vel]
init_state = np.array([0.0, 0.1, np.pi, 1.0])
env.setState(init_state)

In [5]:
# run simulation
num_steps = 100
states = []
for _ in range(num_steps):
    env.performAction(action=0.0)
    # env.remap_angle()
    states.append(env.getState().copy())

states = np.array(states)

In [ ]:
# plot the state trajectory
time = [i * env.delta_time for i in range(num_steps)]
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
title = "Small Oscillation about Stable Equilibirum"


for i in range(4):
    plt.figure(figsize=(6, 4))
    plt.plot(time, states[:, i])
    plt.title(title)
    plt.ylabel(labels[i])
    plt.xlabel('Time (seconds)')
    plt.grid(True)
    # plt.savefig(f"1.1_smallosc_{labels[i].replace(' ', '_').lower()}_vs_time.png")

plt.figure(figsize=(6, 4))
plt.plot(states[:, 1], states[:, 3])
plt.xlabel('Cart Velocity')
plt.ylabel('Pole Angular Velocity')
plt.title(title)
plt.grid(True)
# plt.savefig("1.1_smallosc_cart_vel_vs_pole_angvel.png")

plt.tight_layout()
plt.show()

### 2D Scan Through Different Initial States

In [88]:
env.mu_p = 0.001
env.delta_time = 0.1

In [105]:
num_scan = 100

cart_pos = np.random.uniform(-30, 30) # no effect on system dynamics, not in the equation of motion
cart_vel = np.random.uniform(-30, 30)
pole_ang = np.random.uniform(-np.pi, np.pi)
pole_vel = np.random.uniform(-20, 20)
# [cart_pos, cart_vel, pole_angle, pole_angular_vel]
random_state = [cart_pos, cart_vel, pole_ang, pole_vel]

cart_pos_scan = np.linspace(-30, 30, num_scan)
cart_vel_scan = np.linspace(-30, 30, num_scan)
pole_ang_scan = np.linspace(-np.pi, np.pi, num_scan)
pole_vel_scan = np.linspace(-20, 20, num_scan)
scan = [cart_pos_scan, cart_vel_scan, pole_ang_scan, pole_vel_scan]
next_states = [[],[],[],[]]

In [23]:
# Create grid of (pole_ang, pole_vel) pairs
PA, PV = np.meshgrid(pole_ang_scan, pole_vel_scan)
PA = PA.ravel()
PV = PV.ravel()

X = []
Y = [[], [], [], []]

for pole_ang, pole_vel in zip(PA, PV):
    init_state = np.array([cart_pos, cart_vel, pole_ang, pole_vel])
    env.setState(init_state.copy())
    env.performAction(action=0.0)
    next_state = env.getState().copy()
    delta = next_state - init_state
    for i in range(4):
        Y[i].append(delta[i])
    X.append([pole_ang, pole_vel])

X = np.array(X)
Y = [np.array(y) for y in Y]

In [ ]:
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
fig, axes = plt.subplots(2, 2, figsize=(7, 5))
axes = axes.ravel()

for i in range(4):
    ax = axes[i]
    tcf = ax.tricontourf(X[:, 0], X[:, 1], Y[i], cmap='viridis')
    ax.set_title(f'Δ {labels[i]}')
    ax.set_xlabel('Initial Pole Angle')
    ax.set_ylabel('Initial Pole Angular Velocity')
    fig.colorbar(tcf, ax=ax)

plt.tight_layout()
# plt.savefig('1.2_change_contour_angvel_vs_angle.png')
plt.show()